Import libraries

In [ ]:
!pip install -q fredapi matplotlib plotly keras keras-tuner --upgrade yfinance==0.1.70

from fredapi import Fred
import yfinance as yf

%matplotlib inline
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
from sklearn.preprocessing import MinMaxScaler
from keras.models import Sequential, load_model
from keras.layers import LSTM, Dense, Dropout, Input
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
from sklearn.model_selection import train_test_split
import keras_tuner
from tensorflow.keras.callbacks import EarlyStopping
import time
from matplotlib.ticker import MaxNLocator

Train LSTM model and plot predictions

In [ ]:
# Parameters
seq_length = 2
target = 'Close'
dayspan = 90

# Function to fetch and process data
def getData(ticker_symbol, startday, endday):
    """
    Fetches stock data and various economic indicators, processes them, and saves the combined data to a CSV file.
    Args:
    ticker_symbol (str): The ticker symbol of the stock to download data for.
    startday (str): The start date for the data in 'YYYY-MM-DD' format.
    endday (str): The end date for the data in 'YYYY-MM-DD' format.
    Returns:
    pd.DataFrame: A DataFrame containing the stock data and economic indicators.
    """
    stock_data = yf.download(ticker_symbol, start=startday, end=endday)
    stock_data['Daily_Return'] = stock_data['Close'].pct_change() * 100
    stock_data = stock_data.bfill()
    stock_data.dropna(inplace=True)

    return stock_data

# Function to create sequences
def create_sequences(data, seq_length):
    X, y = [], []
    for i in range(len(data) - seq_length):
        X.append(data[i:i+seq_length])
        y.append(data[i+seq_length])
    return np.array(X), np.array(y)

data = getData('BAC', '2022-01-01', '2023-11-05')

# Split data into train and test sets
train = data.iloc[:-dayspan]
test = data.tail(dayspan)
train.reset_index(drop=True, inplace=True)

# Define target column and features
target_col = target
features = train.columns

# Scale data
scaler = MinMaxScaler()
scaler1 = MinMaxScaler()

X_train = scaler.fit_transform(train[features].values)
y_train = scaler1.fit_transform(train[target_col].values.reshape(-1, 1))
X_test = scaler.transform(test[features].values)
y_test = scaler1.transform(test[target_col].values.reshape(-1, 1))

# Create sequences
X_train_seq, y_train_seq = create_sequences(X_train, seq_length)
X_test_seq, y_test_seq = create_sequences(X_test, seq_length)
dxx, dyy = create_sequences(X_test, seq_length)

# Build the model
model = Sequential()
model.add(Input(shape=((seq_length, len(features)))))
model.add(LSTM(75, activation='relu'))
model.add(Dense(64, activation='relu'))
model.add(Dense(1))

# Compile the model
opt = tf.keras.optimizers.Adam(learning_rate=0.005)
model.compile(optimizer=opt, loss='mean_squared_error', metrics=['accuracy'])

# Train the model with early stopping
early_stop = EarlyStopping(monitor='val_accuracy', patience=50, restore_best_weights=True)
history = model.fit(X_train_seq, y_train_seq, epochs=500, batch_size=128, validation_data=(X_test_seq, y_test_seq), callbacks=[early_stop],verbose=0)

# Inverse transform predictions
scale = MinMaxScaler()
scale.min_, scale.scale_ = scaler.min_[0], scaler.scale_[0]
predbest = scale.inverse_transform(model.predict(dxx)).reshape(-1, 1)
real_prices = test[target_col].values[:-seq_length]

# Plot predicted vs real prices
days = list(range(1, dayspan - seq_length + 1))

plt.figure(figsize=(12, 6))
plt.plot(days, real_prices[0] * predbest / max(predbest), label="Predicted Prices", marker='o', linestyle='-', color='b')
plt.plot(days, real_prices, label="Real Prices", marker='x', linestyle='-', color='g')
plt.xlabel("Days")
plt.ylabel("Price")
plt.title("Predicted vs. Real Prices")
plt.legend()
plt.grid(True)
plt.show()

plt.figure(figsize=(12, 6))
plt.plot(days, predbest, label="Predicted Prices", marker='o', linestyle='-', color='b')
plt.plot(days, real_prices, label="Real Prices", marker='x', linestyle='-', color='g')
plt.xlabel("Days")
plt.ylabel("Price")
plt.title("Predicted vs. Real Prices")
plt.legend()
plt.grid(True)
plt.show()